# Libraries

In [1]:
from pandas.io.json import json_normalize
from urllib.request import urlopen,Request
from os import path
import json
import os
import pandas as pd
import requests
import sqlalchemy
import sqlite3
import csv

# Scrapers developed to pull the data via API

## Movies Dataset

### Documentation: https://developers.themoviedb.org/3/movies/get-movie-details

### Description of fields:

- **budget**:
    - budget expressed in dollars
- **id**:
    - movie ID assigned by themoviedb.org
- **imdb_id**:
    - IMDB ID of the movie
- **original_language**:
    - language in which movie was originally directed
- **original_title**:
    - original title of the movie
- **overview**:
    - a brief description of the movie
- **popularity**:
    - popularity score calculated by themoviedb.org
- **poster_path**:
     - URL of the poster image
- **release_date**:
    - release date of the movie
- **revenue**:
    - revenue expressed in dollars
- **runtime**:
    - runtime of the movie in minutes
- **status**:
    - status of the movie
- **title**:
    - official title of the movie
- **vote_average**:
     - average rating of the movie calculated by themoviedb.org
- **vote_count**:
     - number of user votes calculated by themoviedb.org

In [12]:
def movies(directory, headers, api_key, stop, start, writing_mode):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
        
    while start < stop:
        response = requests.get('https://api.themoviedb.org/3/movie/' + str(start) + '?api_key=' + api_key)
        data = response.json()
        filtered_data = {k : v for k, v in filter(lambda t: t[0] in headers, data.items())}
        df = pd.DataFrame(filtered_data, index = [0])
                 
        with open('movies.csv', str(writing_mode)) as f:
            df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
        print(start)
        start += 1
        
    header_to_pass = ["idx"] + headers
    
    df = pd.read_csv(directory + 'movies.csv', header = None, names = header_to_pass, error_bad_lines = False)
    df.rename(columns={0: header_to_pass[0], 1: header_to_pass[1], 2: header_to_pass[2], 3: header_to_pass[3], 4: header_to_pass[4], 5: header_to_pass[5], 6: header_to_pass[6], 7: header_to_pass[7], 8: header_to_pass[8], 9: header_to_pass[9], 10: header_to_pass[10], 11: header_to_pass[11], 12: header_to_pass[12], 13: header_to_pass[13], 14: header_to_pass[14], 15: header_to_pass[15]}, inplace=True)
    df.to_csv(directory + '/backup/movies.csv', index = False)

    with open(directory + '/backup/movies.csv', 'r') as table:
        dr = csv.DictReader(table, delimiter = ',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]], i[header_to_pass[3]], i[header_to_pass[4]] , i[header_to_pass[5]], i[header_to_pass[6]], i[header_to_pass[7]], i[header_to_pass[8]], i[header_to_pass[9]], i[header_to_pass[10]], i[header_to_pass[11]], i[header_to_pass[12]], i[header_to_pass[13]], i[header_to_pass[14]], i[header_to_pass[15]] ) for i in dr]

        
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    cursor.execute('''
    DROP TABLE IF EXISTS movies
    ''')
    con.commit()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS movies(idx STR, budget INT, id STR, imdb_id STR, original_language STR, original_title STR, overview STR,
                  popularity FLOAT, poster_path STR, release_date DATE, revenue FLOAT, runtime INT, status STR, title STR, vote_average FLOAT, vote_count INT)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO movies VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);", to_db)
    con.commit()
    con.close()


directory = '/home/konrad/movies/'
headers = ["budget", "id","imdb_id", "original_language", "original_title", "overview", "popularity", "poster_path", "release_date","revenue","runtime","status","title","vote_average","vote_count"]
api_key = 'e4f5b3b79e2ab374455c1c1e1d0ca014'
stop = 72501
start = 72500
writing_mode = 'a'

movies(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
72500


## Genres Dataset

### Documentation: https://developers.themoviedb.org/3/movies/get-movie-details

### Description of fields:

- **id**:
    - ID of genre
- **name**:
    - genre name
- **imdb_id**:
    - IMDB ID of movie

In [13]:
def genres(directory, headers, api_key, stop, start, writing_mode):
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
        
    while start < stop:
        response = requests.get('https://api.themoviedb.org/3/movie/' + str(start) +'?api_key=' +  api_key)
        data = response.json()
        
        if 'The resource you requested could not be found.' in data.values():
            start += 1
            
        else:            
            data_normalized = json_normalize(data, "genres", ["imdb_id"])
            df = pd.DataFrame(data_normalized, index = [0])
            
            with open('genres.csv', str(writing_mode)) as f:
                df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
            print(start)
            
        start += 1

    header_to_pass = ["idx"] + headers

    df = pd.read_csv(directory + 'genres.csv', header=None, names=header_to_pass, error_bad_lines=False)
    df.rename(columns={0: header_to_pass[0], 1: header_to_pass[1], 2: header_to_pass[2], 3: header_to_pass[3]}, inplace=True)
    df.to_csv(directory + '/backup/genres.csv', index=False)

    with open(directory + '/backup/genres.csv', 'r') as table:
        dr = csv.DictReader(table, delimiter=',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]], i[header_to_pass[3]]) for i in dr]

    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    cursor.execute('''
    DROP TABLE IF EXISTS genres
    ''')
    con.commit()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS genres(idx STR, id STR, name STR, imdb_id STR)
    ''')
    con.commit()

    cursor.executemany("INSERT INTO genres VALUES (?,?,?,?);", to_db)
    con.commit()
    con.close()
    

    
directory = '/home/konrad/movies/'
headers = ["id", "name", "imdb_id"]
api_key = '69d2d11ea34cd0ff95c454c73786a6a7'
stop = 72501
start = 72500
writing_mode = 'a'

genres(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
72500


## Production Countries Dataset

### Documentation: https://developers.themoviedb.org/3/movies/get-movie-details

### Description of fields:

- **iso_3166_1**:
    - ID of country
- **name**:
    - country name
- **imdb_id**:
    - IMDB ID of the movie

In [24]:
def productionCountries(directory, headers, api_key, stop, start, writing_mode):

    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
        
    while start < stop:
        response = requests.get('https://api.themoviedb.org/3/movie/'+ str(start) +'?api_key='+  api_key)
        data = response.json()
        
        if 'The resource you requested could not be found.' in data.values():
            start += 1
            
        else:
            data_normalized = json_normalize(data,"production_countries",["imdb_id"])
            df = pd.DataFrame(data_normalized, index = [0])

            with open('productioncountries.csv', str(writing_mode)) as f:
                df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
            print(start)
        
        start += 1

    df = pd.read_csv(directory + '/productioncountries.csv', header = None, names = headers, error_bad_lines = False)
    df.rename(columns={0: headers[0], 1: headers[1], 2: headers[2]}, inplace = True)
    df.to_csv(directory + 'backup/productioncountries.csv', index = False)

    with open(directory + '/backup/productioncountries.csv', 'r') as table:
        dr = csv.DictReader(table, delimiter = ',')
        to_db = [(i[headers[0]], i[headers[1]], i[headers[2]]) for i in dr]

    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    cursor.execute('''
    DROP TABLE IF EXISTS productioncountries
    ''')
    con.commit()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS productioncountries(iso_3166_1 STR, name STR, imdb_id STR)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO productioncountries VALUES (?,?,?);", to_db)
    con.commit()
    con.close()

directory = '/home/konrad/movies/'
headers = ["iso_3166_1", "name", "imdb_id"]
api_key = 'ab9bd0e209dd5c8bf42b028a63bc4e29'
stop = 199710
start = 199709
writing_mode = 'a'

productionCountries(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
199704
199705
199708
199709


##  Keywords Dataset

### Documentation: https://developers.themoviedb.org/3/movies/get-movie-keywords

### Description of fields:

- **keywords**:
    - tags used to describe a movie
- **id**:
    - movie ID assigned by themoviedb.org

In [4]:
def keywords(directory, headers, api_key, stop, start, writing_mode):

    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
          
    df = pd.read_sql_query("SELECT imdb_id FROM (SELECT DISTINCT imdb_id FROM movies) WHERE imdb_id <> ''", con)
    l = df['imdb_id'].tolist()
    editedList = l[start:stop]

    for i in editedList:
        response = requests.get("https://api.themoviedb.org/3/movie/" + i + "/keywords?api_key=" + api_key)
        data = response.json()
        data_normalized = json_normalize(data, "keywords", ["id"], meta_prefix = "Movie.")
        df = pd.DataFrame(data_normalized)

        with open(directory + 'keywords.csv', str(writing_mode)) as f:
            df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
        print(start) 
        start += 1
        
    header_to_pass = ["idx"] + headers
    
    df = pd.read_csv(directory + 'keywords.csv', header = None, names = header_to_pass, error_bad_lines = False)
    df.rename(columns={0: header_to_pass[0], 1: header_to_pass[1], 2: header_to_pass[2]}, inplace = True)
    df.to_csv(directory + '/backup/keywords.csv', index = False)

    with open(directory + '/backup/keywords.csv','r') as table:
        dr = csv.DictReader(table, delimiter = ',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]]) for i in dr]
        
    
    cursor.execute('''
        DROP TABLE IF EXISTS keywords
    ''')
    con.commit()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS keywords(idx STR, keywords STR, id STR)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO keywords VALUES (?,?,?);", to_db)
    con.commit()
    con.close()
    

directory = '/home/konrad/movies/'
headers = ["keywords", "id"]
api_key = 'e4f5b3b79e2ab374455c1c1e1d0ca014'
stop = 45751
start = 45750
writing_mode = 'a'

keywords(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
45750


## Cast Dataset

### Documentation: https://developers.themoviedb.org/3/movies/get-movie-credits

### Description of fields:

- **cast_id**:
    - ID of cast member assigned by themoviedb.org
- **character**:
    - character name
- **credit_id**:
    - ID of credit's entry assigned by themoviedb.org
- **imgenderdb_id**:
    - gender ID
- **id**:
    - ID of a person assigned by themoviedb.org
- **name**:
    - person name
- **Movie.order**:
    - order of showing up in the movie
- **profile_path**:
    - URL of person image
- **Movie.id**:
    - movie ID assigned by themoviedb.org

In [15]:
def cast(directory, headers, api_key, stop, start, writing_mode):

    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    df = pd.read_sql_query("SELECT imdb_id FROM (SELECT DISTINCT imdb_id FROM movies) WHERE imdb_id <> ''", con)
    l = df['imdb_id'].tolist()
    editedList = l[start:stop]

    for i in editedList:
        response = requests.get("https://api.themoviedb.org/3/movie/" + i + "/casts?api_key=" + api_key)
        data = response.json()
        data_normalized = json_normalize(data, "cast", ["id"], meta_prefix = "Movie.")
        df = pd.DataFrame(data_normalized)

        with open(directory + 'cast.csv', str(writing_mode)) as f:
            df.to_csv(f, header = False, sep = ',', encoding = 'utf-8')
        
        print(start)    
        start += 1
        
    header_to_pass = ["idx"] + headers

    df = pd.read_csv(directory + 'cast.csv', header = None, names = header_to_pass, error_bad_lines = False)
    df.rename(columns = {0: header_to_pass[0], 1:  header_to_pass[1], 2:  header_to_pass[2], 3:  header_to_pass[3], 4:  header_to_pass[4], 5:  header_to_pass[5], 6:  header_to_pass[6], 7:  header_to_pass[7], 8:  header_to_pass[8], 9:  header_to_pass[9]}, inplace=True)
    df.to_csv(directory + '/backup/cast.csv', index = False)

    with open(directory + '/backup/cast.csv','r') as table:
        dr = csv.DictReader(table, delimiter = ',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]], i[header_to_pass[3]], i[header_to_pass[4]], i[header_to_pass[5]], i[header_to_pass[6]], i[header_to_pass[7]], i[header_to_pass[8]], i[header_to_pass[9]]) for i in dr]


    cursor.execute('''
        DROP TABLE IF EXISTS cast
    ''')
    con.commit()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS cast(idx STR, cast_id STR, character STR, credit_id STR, gender STR, id STR, name STR, "Movie.order" STR, profile_path STR, "Movie.id" STR)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO cast VALUES (?,?,?,?,?,?,?,?,?,?);", to_db)
    con.commit()
    con.close()
    
        

directory = '/home/konrad/movies/'
headers = ["cast_id", "character", "credit_id", "imgenderdb_id", "id", "name", "Movie.order", "profile_path", "Movie.id"]
api_key = 'ab9bd0e209dd5c8bf42b028a63bc4e29'
stop = 25213
start = 25212
writing_mode = 'a'

cast(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies
25212


## Crew Dataset

### Documentation: https://developers.themoviedb.org/3/movies/get-movie-credits

### Description of fields:

- **credit_id**:
    - ID of entry assigned by themoviedb.org
- **department**:
    - name of department person worked in
- **gender**:
    - gender name
- **id**:
    - ID of a person assigned by themoviedb.org
- **job**:
    - role name
- **name**:
    - person name
- **profile_path**:
    - URL of person image
- **Movie.id**:
    - movie ID assigned by themoviedb.org

In [11]:
def crew(directory, headers, api_key, stop, start, writing_mode):

    if not os.path.exists(directory):
        os.makedirs(directory)
        os.chdir(directory)
    else:
        os.chdir(directory)
    print(os.getcwd())
    
    con = sqlite3.connect(directory + 'movies.sqlite')
    cursor = con.cursor()
    
    df = pd.read_sql_query("SELECT imdb_id FROM (SELECT DISTINCT imdb_id FROM movies) WHERE imdb_id <> ''", con)
    l = df['imdb_id'].tolist()
    editedList = l[start:stop]

    for i in editedList:
        response = requests.get("https://api.themoviedb.org/3/movie/" + i + "/casts?api_key=" + api_key)
        data = response.json()
        data_normalized = json_normalize(data,"crew", ["id"], meta_prefix = "Movie.")
        df = pd.DataFrame(data_normalized)
                 
        with open('crew.csv', str(writing_mode)) as f:
            df.to_csv(f, header=False, sep=',', encoding='utf-8')
        
        print(start)    
        start += 1
        
    header_to_pass = ["idx"] + headers
    
    df = pd.read_csv(directory + 'crew.csv', header = None, names = header_to_pass, error_bad_lines = False)
    df.rename(columns = {0: header_to_pass[0], 1: header_to_pass[1] ,2: header_to_pass[2], 3: header_to_pass[3], 4: header_to_pass[4], 5: header_to_pass[5], 6: header_to_pass[6], 7: header_to_pass[7], 8: header_to_pass[8]}, inplace = True)
    df.to_csv(directory + '/backup/crew.csv', index = False)

    with open(directory + '/backup/crew.csv','r') as table:
        dr = csv.DictReader(table, delimiter=',')
        to_db = [(i[header_to_pass[0]], i[header_to_pass[1]], i[header_to_pass[2]], i[header_to_pass[3]], i[header_to_pass[4]], i[header_to_pass[5]], i[header_to_pass[6]], i[header_to_pass[7]], i[header_to_pass[8]]) for i in dr]

    
    cursor.execute('''
    DROP TABLE IF EXISTS crew
    ''')
    con.commit()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS crew(idx STR, credit_id STR, department STR, gender STR, id STR, job STR, name STR, profile_path STR, "Movie.id" STR)
    ''')
    con.commit()
    
    cursor.executemany("INSERT INTO crew VALUES (?,?,?,?,?,?,?,?,?);", to_db)
    con.commit()
    
    

directory = '/home/konrad/movies/'
headers = ["credit_id", "department", "gender", "id", "job", "name", "profile_path", "Movie.id"]
api_key = '69d2d11ea34cd0ff95c454c73786a6a7'
stop = 72500
start = 25182
writing_mode = 'a'
        
crew(directory, headers, api_key, stop, start, writing_mode)

/home/konrad/movies


### Exemplary API queries and outputs stored in JSON format

https://api.themoviedb.org/3/movie/11?api_key=e4f5b3b79e2ab374455c1c1e1d0ca014&language=en-US

In [ ]:
{
    "adult": false,
    "backdrop_path": "/4iJfYYoQzZcONB9hNzg0J0wWyPH.jpg",
    "belongs_to_collection": {
        "id": 10,
        "name": "Star Wars Collection",
        "poster_path": "/iTQHKziZy9pAAY4hHEDCGPaOvFC.jpg",
        "backdrop_path": "/d8duYyyC9J5T825Hg7grmaabfxQ.jpg"
    },
    "budget": 11000000,
    "genres": [
        {
            "id": 12,
            "name": "Adventure"
        },
        {
            "id": 28,
            "name": "Action"
        },
        {
            "id": 878,
            "name": "Science Fiction"
        }
    ],
    "homepage": "http://www.starwars.com/films/star-wars-episode-iv-a-new-hope",
    "id": 11,
    "imdb_id": "tt0076759",
    "original_language": "en",
    "original_title": "Star Wars",
    "overview": "Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.",
    "popularity": 35.013,
    "poster_path": "/btTdmkgIvOi0FFip1sPuZI2oQG6.jpg",
    "production_companies": [
        {
            "id": 1,
            "logo_path": "/o86DbpburjxrqAzEDhXZcyE8pDb.png",
            "name": "Lucasfilm",
            "origin_country": "US"
        },
        {
            "id": 25,
            "logo_path": "/qZCc1lty5FzX30aOCVRBLzaVmcp.png",
            "name": "20th Century Fox",
            "origin_country": "US"
        }
    ],
    "production_countries": [
        {
            "iso_3166_1": "US",
            "name": "United States of America"
        }
    ],
    "release_date": "1977-05-25",
    "revenue": 775398007,
    "runtime": 121,
    "spoken_languages": [
        {
            "iso_639_1": "en",
            "name": "English"
        }
    ],
    "status": "Released",
    "tagline": "A long time ago in a galaxy far, far away...",
    "title": "Star Wars",
    "video": false,
    "vote_average": 8.2,
    "vote_count": 10443
}

https://api.themoviedb.org/3/movie/11/credits?api_key=e4f5b3b79e2ab374455c1c1e1d0ca014

In [ ]:
{
    "id": 11,
    "cast": [
        {
            "cast_id": 3,
            "character": "Luke Skywalker",
            "credit_id": "52fe420dc3a36847f8000441",
            "gender": 2,
            "id": 2,
            "name": "Mark Hamill",
            "order": 0,
            "profile_path": "/fk8OfdReNltKZqOk2TZgkofCUFq.jpg"
        },
        {
            "cast_id": 4,
            "character": "Han Solo",
            "credit_id": "52fe420dc3a36847f8000445",
            "gender": 2,
            "id": 3,
            "name": "Harrison Ford",
            "order": 1,
            "profile_path": "/7CcoVFTogQgex2kJkXKMe8qHZrC.jpg"
        },
        {
            "cast_id": 5,
            "character": "Princess Leia Organa",
            "credit_id": "52fe420dc3a36847f8000449",
            "gender": 1,
            "id": 4,
            "name": "Carrie Fisher",
            "order": 2,
            "profile_path": "/rfJtncHewKVnHjqpIZvjn24ESeC.jpg"
        },
        {
            "cast_id": 15,
            "character": "Chewbacca",
            "credit_id": "52fe420dc3a36847f800047b",
            "gender": 2,
            "id": 24343,
            "name": "Peter Mayhew",
            "order": 3,
            "profile_path": "/rKIBv08O63pyC6PixjyhrjNB8Uh.jpg"
        },
        {
            "cast_id": 7,
            "character": "C-3PO",
            "credit_id": "52fe420dc3a36847f8000451",
            "gender": 2,
            "id": 6,
            "name": "Anthony Daniels",
            "order": 4,
            "profile_path": "/cljvryjb3VwTsNR7fjQKjNPMaBB.jpg"
        },
        {
            "cast_id": 16,
            "character": "Darth Vader (performer)",
            "credit_id": "52fe420dc3a36847f800047f",
            "gender": 2,
            "id": 24342,
            "name": "David Prowse",
            "order": 5,
            "profile_path": "/cJtmBVrjYwawh2cCiAfZkEjPeqc.jpg"
        },
        {
            "cast_id": 17,
            "character": "Darth Vader (voice)",
            "credit_id": "52fe420dc3a36847f8000483",
            "gender": 2,
            "id": 15152,
            "name": "James Earl Jones",
            "order": 6,
            "profile_path": "/oqMPIsXrl9SZkRfIKN08eFROmH6.jpg"
        },
        {
            "cast_id": 6,
            "character": "Grand Moff Tarkin",
            "credit_id": "52fe420dc3a36847f800044d",
            "gender": 2,
            "id": 5,
            "name": "Peter Cushing",
            "order": 7,
            "profile_path": "/l0grZXcjqctESg7h6Jdp33pA3QG.jpg"
        },
        {
            "cast_id": 14,
            "character": "Obi-Wan \"Ben\" Kenobi",
            "credit_id": "52fe420dc3a36847f8000477",
            "gender": 2,
            "id": 12248,
            "name": "Alec Guinness",
            "order": 8,
            "profile_path": "/nv3ppxgUQJytFGXZNde4f9ZlshB.jpg"
        },
        {
            "cast_id": 18,
            "character": "Uncle Owen",
            "credit_id": "52fe420dc3a36847f8000487",
            "gender": 2,
            "id": 33032,
            "name": "Phil Brown",
            "order": 9,
            "profile_path": "/exkyN66HiZWJDmpcOza2hWoswOo.jpg"
        },
        {
            "cast_id": 19,
            "character": "Aunt Beru",
            "credit_id": "52fe420dc3a36847f800048b",
            "gender": 1,
            "id": 131625,
            "name": "Shelagh Fraser",
            "order": 10,
            "profile_path": "/xNfiibBvknHztEnL0g7dcdrxOKq.jpg"
        },
        {
            "cast_id": 8,
            "character": "R2-D2",
            "credit_id": "52fe420dc3a36847f8000455",
            "gender": 2,
            "id": 130,
            "name": "Kenny Baker",
            "order": 11,
            "profile_path": "/sdd9rgifNF9C51RejG7sUGU8Bka.jpg"
        },
        {
            "cast_id": 24,
            "character": "Chief Jawa",
            "credit_id": "52fe420dc3a36847f800049f",
            "gender": 2,
            "id": 132538,
            "name": "Jack Purvis",
            "order": 12,
            "profile_path": "/tuFTY1jhlEgZm3vM80KdAEvHwNI.jpg"
        },
        {
            "cast_id": 26,
            "character": "General Willard",
            "credit_id": "52fe420dc3a36847f80004a7",
            "gender": 2,
            "id": 69249,
            "name": "Eddie Byrne",
            "order": 13,
            "profile_path": "/mSwNawI6Ou8m99Y05WjctoTWYUK.jpg"
        },
        {
            "cast_id": 33,
            "character": "General Taggi",
            "credit_id": "52fe420dc3a36847f80004c3",
            "gender": 2,
            "id": 42570,
            "name": "Don Henderson",
            "order": 14,
            "profile_path": "/qeOAWEiZ4cXddRziyaJQ2Mt5Mpm.jpg"
        },
        {
            "cast_id": 25,
            "character": "General Dodonna",
            "credit_id": "52fe420dc3a36847f80004a3",
            "gender": 2,
            "id": 216087,
            "name": "Alex McCrindle",
            "order": 15,
            "profile_path": "/LTLiQPMnKESxGVHhZ7n2qPAuMi.jpg"
        },
        {
            "cast_id": 34,
            "character": "General Motti",
            "credit_id": "52fe420dc3a36847f80004c7",
            "gender": 2,
            "id": 12829,
            "name": "Richard LeParmentier",
            "order": 16,
            "profile_path": "/eyQiSGCHIcmdU2gqKWZbneQEvFj.jpg"
        },
        {
            "cast_id": 23,
            "character": "Red Leader",
            "credit_id": "52fe420dc3a36847f800049b",
            "gender": 2,
            "id": 47401,
            "name": "Drewe Henley",
            "order": 17,
            "profile_path": "/C28FmnpDyhI9BwD6YjagAe1U53.jpg"
        },
        {
            "cast_id": 27,
            "character": "Red Two (Wedge Antilles)",
            "credit_id": "52fe420dc3a36847f80004ab",
            "gender": 2,
            "id": 47698,
            "name": "Denis Lawson",
            "order": 18,
            "profile_path": "/nxPR7KIlJ4CPPI2hniTUk6bu9fA.jpg"
        },
        {
            "cast_id": 28,
            "character": "Red Three (Biggs Darklighter)",
            "credit_id": "52fe420dc3a36847f80004af",
            "gender": 2,
            "id": 17356,
            "name": "Garrick Hagon",
            "order": 19,
            "profile_path": "/lZYitsCPzlwevNuHzqaSZMQiuUa.jpg"
        },
        {
            "cast_id": 29,
            "character": "Red Four (John \"D\")",
            "credit_id": "52fe420dc3a36847f80004b3",
            "gender": 2,
            "id": 162432,
            "name": "Jack Klaff",
            "order": 20,
            "profile_path": "/6l21oFayFKyyuBEELJEaj3veo21.jpg"
        },
        {
            "cast_id": 21,
            "character": "Red Six (Porkins)",
            "credit_id": "52fe420dc3a36847f8000493",
            "gender": 2,
            "id": 663,
            "name": "William Hootkins",
            "order": 21,
            "profile_path": "/lGPSg64fsqbWS5PUFKsUKLNOqsx.jpg"
        },
        {
            "cast_id": 35,
            "character": "Commander #1",
            "credit_id": "52fe420dc3a36847f80004cb",
            "gender": 2,
            "id": 79489,
            "name": "Leslie Schofield",
            "order": 22,
            "profile_path": "/r1WQsrbi1XkbfpORgrWTDNGQCKD.jpg"
        },
        {
            "cast_id": 128,
            "character": "Stormtrooper",
            "credit_id": "586570c7c3a36852d201e865",
            "gender": 2,
            "id": 1729623,
            "name": "Michael Leader",
            "order": 23,
            "profile_path": null
        },
        {
            "cast_id": 30,
            "character": "Gold Leader",
            "credit_id": "52fe420dc3a36847f80004b7",
            "gender": 2,
            "id": 58475,
            "name": "Angus MacInnes",
            "order": 24,
            "profile_path": "/qftkol8hj7yBBP3KCxRWYkhRyLC.jpg"
        },
        {
            "cast_id": 126,
            "character": "Gold Two",
            "credit_id": "569792c4c3a3683b040021fc",
            "gender": 2,
            "id": 151819,
            "name": "Jeremy Sinden",
            "order": 25,
            "profile_path": null
        },
        {
            "cast_id": 32,
            "character": "Gold Five",
            "credit_id": "52fe420dc3a36847f80004bf",
            "gender": 2,
            "id": 202276,
            "name": "Graham Ashley",
            "order": 26,
            "profile_path": "/wp02ruOjX8AiGMrRD8QEBljgnlA.jpg"
        },
        {
            "cast_id": 36,
            "character": "Red Two, Wedge Antilles (voice) (uncredited)",
            "credit_id": "52fe420dc3a36847f80004cf",
            "gender": 2,
            "id": 1216947,
            "name": "David Ankrum",
            "order": 27,
            "profile_path": "/vo6JMA38exMSSbyQ3K0YCBwBrWT.jpg"
        },
        {
            "cast_id": 37,
            "character": "Boba Fett (special edition) (uncredited)",
            "credit_id": "52fe420dc3a36847f80004d3",
            "gender": 2,
            "id": 1271058,
            "name": "Mark Austin",
            "order": 28,
            "profile_path": "/3Zocn38GPVYwWSgVEE3jKJvKyNT.jpg"
        },
        {
            "cast_id": 38,
            "character": "Stormtrooper (voice) (uncredited)",
            "credit_id": "52fe420dc3a36847f80004d7",
            "gender": 2,
            "id": 3044,
            "name": "Scott Beach",
            "order": 29,
            "profile_path": "/gkt4TpoRR75eTddsny3Qvofe6TY.jpg"
        },
        {
            "cast_id": 39,
            "character": "Stormtrooper (uncredited)",
            "credit_id": "52fe420ec3a36847f80004db",
            "gender": 2,
            "id": 1271059,
            "name": "Lightning Bear",
            "order": 30,
            "profile_path": "/RWJgw1QvH1rgIzGTM6QYwRiQRC.jpg"
        },
        {
            "cast_id": 40,
            "character": "Cantina Alien (uncredited)",
            "credit_id": "52fe420ec3a36847f80004df",
            "gender": 2,
            "id": 1271060,
            "name": "Jon Berg",
            "order": 31,
            "profile_path": "/q4IxPRLu82E3ppEw02GeejaNGeJ.jpg"
        },
        {
            "cast_id": 41,
            "character": "Cantina Alien (uncredited)",
            "credit_id": "52fe420ec3a36847f80004e3",
            "gender": 2,
            "id": 1271061,
            "name": "Doug Beswick",
            "order": 32,
            "profile_path": "/iKnyfUrS410O4yQShOAqVTw2SyU.jpg"
        },
        {
            "cast_id": 42,
            "character": "Greedo (uncredited)",
            "credit_id": "52fe420ec3a36847f80004e7",
            "gender": 2,
            "id": 199356,
            "name": "Paul Blake",
            "order": 33,
            "profile_path": "/WSO4C7YdURE1thtj2MPkWSKD6o.jpg"
        },
        {
            "cast_id": 43,
            "character": "Nabrun Leids (uncredited)",
            "credit_id": "52fe420ec3a36847f80004eb",
            "gender": 0,
            "id": 1271062,
            "name": "Janice Burchette",
            "order": 34,
            "profile_path": null
        },
        {
            "cast_id": 44,
            "character": "Wuher (uncredited)",
            "credit_id": "52fe420ec3a36847f80004ef",
            "gender": 2,
            "id": 1271063,
            "name": "Ted Burnett",
            "order": 35,
            "profile_path": "/A1GOhCu6LH4fg1VlyOT08NjWmJU.jpg"
        },
        {
            "cast_id": 45,
            "character": "Drifter (Red 12) (uncredited)",
            "credit_id": "52fe420ec3a36847f80004f3",
            "gender": 2,
            "id": 1271064,
            "name": "John Chapman",
            "order": 36,
            "profile_path": "/oqqR2ylj8CyjlIaSizHlhQlZ1PV.jpg"
        },
        {
            "cast_id": 46,
            "character": "Cantina Patron (uncredited)",
            "credit_id": "52fe420ec3a36847f80004f7",
            "gender": 2,
            "id": 1271065,
            "name": "Gilda Cohen",
            "order": 37,
            "profile_path": "/uSByRJBieeMpIwg4SeqB8XFCy7x.jpg"
        },
        {
            "cast_id": 47,
            "character": "Stormtrooper (uncredited)",
            "credit_id": "52fe420ec3a36847f80004fb",
            "gender": 2,
            "id": 1178140,
            "name": "Tim Condren",
            "order": 38,
            "profile_path": "/b2LuPeiNkiN7YT0mpJ1O0C2BV58.jpg"
        },
        {
            "cast_id": 48,
            "character": "Wioslea (uncredited)",
            "credit_id": "52fe420ec3a36847f80004ff",
            "gender": 0,
            "id": 186229,
            "name": "Barry Copping",
            "order": 39,
            "profile_path": null
        },
        {
            "cast_id": 49,
            "character": "Dr. Evazan (uncredited)",
            "credit_id": "52fe420ec3a36847f8000503",
            "gender": 2,
            "id": 1271066,
            "name": "Alfie Curtis",
            "order": 40,
            "profile_path": "/5jKHKbIF1cEWeG2sPAzHScgGW7n.jpg"
        },
        {
            "cast_id": 50,
            "character": "Cantina Patron (uncredited)",
            "credit_id": "52fe420ec3a36847f8000507",
            "gender": 0,
            "id": 1271067,
            "name": "Robert Davies",
            "order": 41,
            "profile_path": null
        },
        {
            "cast_id": 51,
            "character": "Greedo (uncredited)",
            "credit_id": "52fe420ec3a36847f800050b",
            "gender": 1,
            "id": 104149,
            "name": "Maria De Aragon",
            "order": 42,
            "profile_path": "/rnaslrjV5ui6cKphksSni3K0TVQ.jpg"
        },
        {
            "cast_id": 52,
            "character": "Hrchek Kal Fas (uncredited)",
            "credit_id": "52fe420ec3a36847f800050f",
            "gender": 0,
            "id": 1271068,
            "name": "Robert A. Denham",
            "order": 43,
            "profile_path": null
        },
        {
            "cast_id": 53,
            "character": "Jawa (uncredited)",
            "credit_id": "52fe420ec3a36847f8000513",
            "gender": 0,
            "id": 1271069,
            "name": "Frazer Diamond",
            "order": 44,
            "profile_path": "/9e3i9TJ0pp5zbUnDljNuvkjhgCW.jpg"
        },
        {
            "cast_id": 54,
            "character": "Stormtrooper / Tusken Raider / Death Star Trooper / Garouf Lafoe (uncredited)",
            "credit_id": "52fe420ec3a36847f8000517",
            "gender": 2,
            "id": 53587,
            "name": "Peter Diamond",
            "order": 45,
            "profile_path": "/rIf04LU2CsdzdvUJghFVVjdWcm6.jpg"
        },
        {
            "cast_id": 55,
            "character": "Jawa (uncredited)",
            "credit_id": "52fe420ec3a36847f800051b",
            "gender": 0,
            "id": 1271070,
            "name": "Warwick Diamond",
            "order": 46,
            "profile_path": "/3E5Ktz0o6k4Xz6iTq2zCVHsxleX.jpg"
        },
        {
            "cast_id": 56,
            "character": "Garindan (uncredited)",
            "credit_id": "52fe420ec3a36847f800051f",
            "gender": 0,
            "id": 1271071,
            "name": "Sadie Eden",
            "order": 47,
            "profile_path": null
        },
        {
            "cast_id": 57,
            "character": "Djas Puhr (uncredited)",
            "credit_id": "52fe420ec3a36847f8000523",
            "gender": 0,
            "id": 1271072,
            "name": "Kim Falkinburg",
            "order": 48,
            "profile_path": null
        },
        {
            "cast_id": 58,
            "character": "Death Star Trooper (uncredited)",
            "credit_id": "52fe420ec3a36847f8000527",
            "gender": 2,
            "id": 202402,
            "name": "Harry Fielder",
            "order": 49,
            "profile_path": "/tVA1eKmQk3RXMLCuDP0cO5r5txJ.jpg"
        },
        {
            "cast_id": 59,
            "character": "Stormtrooper with Binoculars (uncredited)",
            "credit_id": "52fe420ec3a36847f800052b",
            "gender": 0,
            "id": 1271073,
            "name": "Ted Gagliano",
            "order": 50,
            "profile_path": "/jpfBK5PYsY13c1gey5HdojwWW8i.jpg"
        },
        {
            "cast_id": 60,
            "character": "Cantina Patron (uncredited)",
            "credit_id": "52fe420ec3a36847f800052f",
            "gender": 0,
            "id": 1271074,
            "name": "Salo Gardner",
            "order": 51,
            "profile_path": "/dEDmkjjpqaNcadl2vVwO6osg8Yv.jpg"
        },
        {
            "cast_id": 61,
            "character": "Death Star Trooper (uncredited)",
            "credit_id": "52fe420ec3a36847f8000533",
            "gender": 0,
            "id": 1195602,
            "name": "Steve Gawley",
            "order": 52,
            "profile_path": "/q0XmjHBKRdWsZfMnP3ks30NdzXb.jpg"
        },
        {
            "cast_id": 62,
            "character": "Kabe (uncredited)",
            "credit_id": "52fe420ec3a36847f8000537",
            "gender": 0,
            "id": 1271075,
            "name": "Barry Gnome",
            "order": 53,
            "profile_path": null
        },
        {
            "cast_id": 63,
            "character": "Kabe / Jawa / GONK Droid (uncredited)",
            "credit_id": "52fe420ec3a36847f800053b",
            "gender": 0,
            "id": 252527,
            "name": "Rusty Goffe",
            "order": 54,
            "profile_path": "/a9yzdfW4H05dn8Bv8VRefYe2cUe.jpg"
        },
        {
            "cast_id": 64,
            "character": "Cantina Patron (uncredited)",
            "credit_id": "52fe420ec3a36847f800053f",
            "gender": 0,
            "id": 1271076,
            "name": "Isaac Grand",
            "order": 55,
            "profile_path": null
        },
        {
            "cast_id": 65,
            "character": "Stormtrooper (special edition) (uncredited)",
            "credit_id": "52fe420ec3a36847f8000543",
            "gender": 0,
            "id": 1271077,
            "name": "Nelson Hall",
            "order": 56,
            "profile_path": "/8t1Fx7haF4OYN7ah57FitSeQDLf.jpg"
        },
        {
            "cast_id": 66,
            "character": "Stormtrooper (uncredited)",
            "credit_id": "52fe420ec3a36847f8000547",
            "gender": 0,
            "id": 1271078,
            "name": "Reg Harding",
            "order": 57,
            "profile_path": "/2saprCiNLI7rpGmNSPjEVvlkxXA.jpg"
        },
        {
            "cast_id": 67,
            "character": "Leia's Rebel Escort (uncredited)",
            "credit_id": "52fe420ec3a36847f800054b",
            "gender": 2,
            "id": 964699,
            "name": "Alan Harris",
            "order": 58,
            "profile_path": "/t7bLuzCIGJWn7FRUVoHHQGzijWo.jpg"
        },
        {
            "cast_id": 68,
            "character": "Stormtrooper (uncredited)",
            "credit_id": "52fe420ec3a36847f800054f",
            "gender": 0,
            "id": 1271079,
            "name": "Frank Henson",
            "order": 59,
            "profile_path": "/dFYRMa53HVxrHahfMMgutssxsMP.jpg"
        },
        {
            "cast_id": 69,
            "character": "Brea Tonnika (uncredited)",
            "credit_id": "52fe420ec3a36847f8000553",
            "gender": 0,
            "id": 1271080,
            "name": "Christine Hewett",
            "order": 60,
            "profile_path": "/67ZqL2PGP2o6uLBOOwzZLikwHHp.jpg"
        },
        {
            "cast_id": 70,
            "character": "Stormtrooper (uncredited)",
            "credit_id": "52fe420ec3a36847f8000557",
            "gender": 0,
            "id": 1271081,
            "name": "Arthur Howell",
            "order": 61,
            "profile_path": "/rQU7GbmvonJN8SGjcbusEb9M1aG.jpg"
        },
        {
            "cast_id": 71,
            "character": "Ponda Baba (uncredited)",
            "credit_id": "52fe420ec3a36847f800055b",
            "gender": 0,
            "id": 1271082,
            "name": "Tommy Ilsley",
            "order": 62,
            "profile_path": null
        },
        {
            "cast_id": 72,
            "character": "Death Star Trooper (uncredited)",
            "credit_id": "52fe420ec3a36847f800055f",
            "gender": 2,
            "id": 4945,
            "name": "Joe Johnston",
            "order": 63,
            "profile_path": "/fbGZo6CG9Z9zKFh8D5wHunyu7gJ.jpg"
        },
        {
            "cast_id": 73,
            "character": "Mosep (uncredited)",
            "credit_id": "52fe420ec3a36847f8000563",
            "gender": 0,
            "id": 1271083,
            "name": "Annette Jones",
            "order": 64,
            "profile_path": null
        },
        {
            "cast_id": 74,
            "character": "Chall Bekan (uncredited)",
            "credit_id": "52fe420ec3a36847f8000567",
            "gender": 0,
            "id": 1148750,
            "name": "Linda Jones",
            "order": 65,
            "profile_path": null
        },
        {
            "cast_id": 75,
            "character": "Solomohal (uncredited)",
            "credit_id": "52fe420ec3a36847f800056b",
            "gender": 0,
            "id": 201344,
            "name": "Joe Kaye",
            "order": 66,
            "profile_path": null
        },
        {
            "cast_id": 76,
            "character": "Stormtrooper (voice) (uncredited)",
            "credit_id": "52fe420ec3a36847f800056f",
            "gender": 0,
            "id": 1271085,
            "name": "Colin Michael Kitchens",
            "order": 67,
            "profile_path": "/8Z7P5PE6HHKR5ekEWLdIFbuBZkb.jpg"
        },
        {
            "cast_id": 77,
            "character": "Jawa (uncredited)",
            "credit_id": "52fe420ec3a36847f8000573",
            "gender": 0,
            "id": 1271086,
            "name": "Melissa Kurtz",
            "order": 68,
            "profile_path": "/8KFIcrGTYoI7twjwerAnQH1eaum.jpg"
        },
        {
            "cast_id": 78,
            "character": "Jawa (uncredited)",
            "credit_id": "52fe420ec3a36847f8000577",
            "gender": 0,
            "id": 1190780,
            "name": "Tiffany L. Kurtz",
            "order": 69,
            "profile_path": "/ttaKrsKhmbpu7sh0vRymmm3qX3S.jpg"
        },
        {
            "cast_id": 79,
            "character": "Daine Jir (uncredited)",
            "credit_id": "52fe420ec3a36847f800057b",
            "gender": 0,
            "id": 190453,
            "name": "Al Lampert",
            "order": 70,
            "profile_path": "/8YLrP1AQTVtP6G1FJnPsiQOOOO5.jpg"
        },
        {
            "cast_id": 80,
            "character": "BoShek (uncredited)",
            "credit_id": "52fe420ec3a36847f800057f",
            "gender": 0,
            "id": 1012562,
            "name": "Anthony Lang",
            "order": 71,
            "profile_path": "/xdrvGrXLIVw65PTdozxHUPDRgFQ.jpg"
        },
        {
            "cast_id": 81,
            "character": "Muftak / Cantina Band Member (uncredited)",
            "credit_id": "52fe420ec3a36847f8000583",
            "gender": 0,
            "id": 1271091,
            "name": "Laine Liska",
            "order": 72,
            "profile_path": null
        },
        {
            "cast_id": 82,
            "character": "Temple Guard / Medal Bearer (uncredited)",
            "credit_id": "52fe420ec3a36847f8000587",
            "gender": 2,
            "id": 1271092,
            "name": "Derek Lyons",
            "order": 73,
            "profile_path": "/oO7dJlNLJhYyqdTsoUQAFXp1UQS.jpg"
        },
        {
            "cast_id": 83,
            "character": "Jawa (uncredited)",
            "credit_id": "52fe420ec3a36847f800058b",
            "gender": 0,
            "id": 1271095,
            "name": "Mahjoub",
            "order": 74,
            "profile_path": null
        },
        {
            "cast_id": 84,
            "character": "Takeel (uncredited)",
            "credit_id": "52fe420ec3a36847f800058f",
            "gender": 0,
            "id": 1271096,
            "name": "Alf Mangan",
            "order": 75,
            "profile_path": "/yZ1jthofTE7NCwcJkGMCSjrFpz6.jpg"
        },
        {
            "cast_id": 85,
            "character": "Stormtrooper (special edition) (uncredited)",
            "credit_id": "52fe420ec3a36847f8000593",
            "gender": 0,
            "id": 19801,
            "name": "Rick McCallum",
            "order": 76,
            "profile_path": "/qfKgg9sNcIOp2hELpEksqI4DDoo.jpg"
        },
        {
            "cast_id": 86,
            "character": "Death Star Gunner (uncredited)",
            "credit_id": "52fe420ec3a36847f8000597",
            "gender": 0,
            "id": 1271102,
            "name": "Grant McCune",
            "order": 77,
            "profile_path": "/dyYcw0CDPRWZP0upMV0UPdCVTZw.jpg"
        },
        {
            "cast_id": 87,
            "character": "Cantina Patron (uncredited)",
            "credit_id": "52fe420ec3a36847f800059b",
            "gender": 0,
            "id": 1271104,
            "name": "Geoffrey Moon",
            "order": 78,
            "profile_path": "/mmSpFa7i6gJVBxETLHyiMLx54ay.jpg"
        },
        {
            "cast_id": 88,
            "character": "Swilla Corey (uncredited)",
            "credit_id": "52fe420ec3a36847f800059f",
            "gender": 0,
            "id": 1271106,
            "name": "Mandy Morton",
            "order": 79,
            "profile_path": "/2lmVJN1qV5zAYDy4DYwRxvW8nCb.jpg"
        },
        {
            "cast_id": 89,
            "character": "Massassi Base Rebel Scout (uncredited)",
            "credit_id": "52fe420ec3a36847f80005a3",
            "gender": 0,
            "id": 1271107,
            "name": "Lorne Peterson",
            "order": 80,
            "profile_path": "/xCIzR3kH76oNJga9gRNAwxPm2yu.jpg"
        },
        {
            "cast_id": 90,
            "character": "Rycar Ryjerd (uncredited)",
            "credit_id": "52fe420ec3a36847f80005a7",
            "gender": 0,
            "id": 1183443,
            "name": "Marcus Powell",
            "order": 81,
            "profile_path": "/cNIpsCHwTl5CCtzaqVSfNjsHQe5.jpg"
        },
        {
            "cast_id": 91,
            "character": "InCom Engineer (uncredited)",
            "credit_id": "52fe420ec3a36847f80005ab",
            "gender": 2,
            "id": 10657,
            "name": "Shane Rimmer",
            "order": 82,
            "profile_path": "/ctrIOcWLjOB5rocS0vVHEjbS1Sx.jpg"
        },
        {
            "cast_id": 92,
            "character": "Leesub Sirln (uncredited)",
            "credit_id": "52fe420ec3a36847f80005af",
            "gender": 0,
            "id": 1271116,
            "name": "Pam Rose",
            "order": 83,
            "profile_path": "/uaFDw1Ksx9ctyDKhoxTw4aFRRtu.jpg"
        },
        {
            "cast_id": 93,
            "character": "Cmdr. Praji (Imperial Officer #2 on rebel ship) (uncredited)",
            "credit_id": "52fe420ec3a36847f80005b3",
            "gender": 2,
            "id": 110319,
            "name": "George Roubicek",
            "order": 84,
            "profile_path": "/ru7WxtpEOkWADyemk3XlK61v5GS.jpg"
        },
        {
            "cast_id": 94,
            "character": "Tawss Khaa (uncredited)",
            "credit_id": "52fe420ec3a36847f80005b7",
            "gender": 0,
            "id": 1271121,
            "name": "Erica Simmons",
            "order": 85,
            "profile_path": null
        },
        {
            "cast_id": 95,
            "character": "Senni Tonnika (uncredited)",
            "credit_id": "52fe420ec3a36847f80005bb",
            "gender": 0,
            "id": 1271122,
            "name": "Angela Staines",
            "order": 86,
            "profile_path": "/ydiU1ozqqeWuSXYMEKyLattGUr0.jpg"
        },
        {
            "cast_id": 96,
            "character": "Cantina Patron (uncredited)",
            "credit_id": "52fe420ec3a36847f80005bf",
            "gender": 0,
            "id": 1271123,
            "name": "George Stock",
            "order": 87,
            "profile_path": null
        },
        {
            "cast_id": 97,
            "character": "Cantina Patron (uncredited)",
            "credit_id": "52fe420ec3a36847f80005c3",
            "gender": 0,
            "id": 1271124,
            "name": "Roy Straite",
            "order": 88,
            "profile_path": "/iTTVUQwq9Jit8BmrDYN0dmZXQjG.jpg"
        },
        {
            "cast_id": 98,
            "character": "Sai'torr Kal Fas (uncredited)",
            "credit_id": "52fe420ec3a36847f80005c7",
            "gender": 0,
            "id": 1271125,
            "name": "Peter Sturgeon",
            "order": 89,
            "profile_path": null
        },
        {
            "cast_id": 99,
            "character": "Lt. Pol Treidum (uncredited)",
            "credit_id": "52fe420ec3a36847f80005cb",
            "gender": 2,
            "id": 187398,
            "name": "Peter Sumner",
            "order": 90,
            "profile_path": "/3BiflFG5cnHYI1Ehn85PhTyCZaf.jpg"
        },
        {
            "cast_id": 100,
            "character": "Cantina Voices (voice) (uncredited)",
            "credit_id": "52fe420ec3a36847f80005cf",
            "gender": 0,
            "id": 1271126,
            "name": "John Sylla",
            "order": 91,
            "profile_path": "/46ef2FOF35ieIFM14A8F8nch85t.jpg"
        },
        {
            "cast_id": 101,
            "character": "Massassi Outpost Announcer / Various Voices (voice) (uncredited)",
            "credit_id": "52fe420ec3a36847f80005d3",
            "gender": 0,
            "id": 390158,
            "name": "Tom Sylla",
            "order": 92,
            "profile_path": "/7oKTyEfvDwS7zOz3wsMU5z51P4P.jpg"
        },
        {
            "cast_id": 22,
            "character": "Lt. Shann Childsen (uncredited)",
            "credit_id": "52fe420dc3a36847f8000497",
            "gender": 2,
            "id": 166258,
            "name": "Malcolm Tierney",
            "order": 93,
            "profile_path": "/fe7Cz6sxTLt9qSQANRpAAaYcPlV.jpg"
        },
        {
            "cast_id": 102,
            "character": "Cantina Alien (uncredited)",
            "credit_id": "52fe420ec3a36847f80005d7",
            "gender": 2,
            "id": 7727,
            "name": "Phil Tippett",
            "order": 94,
            "profile_path": "/2uQ0B7fN5cDQk17J1X3pxDSf9y.jpg"
        },
        {
            "cast_id": 103,
            "character": "Del Goren (uncredited)",
            "credit_id": "52fe420ec3a36847f80005db",
            "gender": 0,
            "id": 184980,
            "name": "Burnell Tucker",
            "order": 95,
            "profile_path": "/kRMCT2aPlZO5Cl5404mRyHEQBt6.jpg"
        },
        {
            "cast_id": 104,
            "character": "Stormtrooper (voice) (uncredited)",
            "credit_id": "52fe420ec3a36847f80005df",
            "gender": 0,
            "id": 160947,
            "name": "Morgan Upton",
            "order": 96,
            "profile_path": "/c6cHfJSxRl6Z9D2BcYNELq9ZwEZ.jpg"
        },
        {
            "cast_id": 105,
            "character": "Stormtrooper (voice) (uncredited)",
            "credit_id": "52fe420ec3a36847f80005e3",
            "gender": 0,
            "id": 161301,
            "name": "Jerry Walter",
            "order": 97,
            "profile_path": "/kxMyDTBi2DpgVnzPgbOJTokpMUy.jpg"
        },
        {
            "cast_id": 106,
            "character": "Jawa (uncredited)",
            "credit_id": "52fe420ec3a36847f80005e7",
            "gender": 0,
            "id": 1271127,
            "name": "Hal Wamsley",
            "order": 98,
            "profile_path": "/4nRNQyY5TyKNpNIUK6Z9JRr3xWw.jpg"
        },
        {
            "cast_id": 107,
            "character": "Greedo (voice) (uncredited)",
            "credit_id": "52fe420ec3a36847f80005eb",
            "gender": 2,
            "id": 159108,
            "name": "Larry Ward",
            "order": 99,
            "profile_path": "/zRqHcr0ySV1IJiDVmpciCokzn3h.jpg"
        },
        {
            "cast_id": 108,
            "character": "Thuku (uncredited)",
            "credit_id": "52fe420ec3a36847f80005ef",
            "gender": 0,
            "id": 1271128,
            "name": "Diana Sadley Way",
            "order": 100,
            "profile_path": null
        },
        {
            "cast_id": 109,
            "character": "Ketwol / Melas (uncredited)",
            "credit_id": "52fe420ec3a36847f80005f3",
            "gender": 0,
            "id": 1271129,
            "name": "Harold Weed",
            "order": 101,
            "profile_path": "/ysocn4XckZRUqZsUp4qwldLCgZk.jpg"
        },
        {
            "cast_id": 110,
            "character": "Stormtrooper (uncredited)",
            "credit_id": "52fe420ec3a36847f80005f7",
            "gender": 2,
            "id": 24278,
            "name": "Bill Weston",
            "order": 102,
            "profile_path": "/apVNA3SRNIrB6gu88nGTEhPewI2.jpg"
        },
        {
            "cast_id": 111,
            "character": "Mos Eisley Citizen (special edition) (uncredited)",
            "credit_id": "52fe420ec3a36847f80005fb",
            "gender": 2,
            "id": 60207,
            "name": "Steve 'Spaz' Williams",
            "order": 103,
            "profile_path": "/zD0Qyhjjg87fDdEDJbqOFsSMtjm.jpg"
        },
        {
            "cast_id": 112,
            "character": "Cantina Patron (uncredited)",
            "credit_id": "52fe420ec3a36847f80005ff",
            "gender": 2,
            "id": 1271131,
            "name": "Fred Wood",
            "order": 104,
            "profile_path": null
        },
        {
            "cast_id": 129,
            "character": "Rebel Pilot, Col. Takbright (uncredited)",
            "credit_id": "588f15b4c3a36860ce00562c",
            "gender": 2,
            "id": 301657,
            "name": "Colin Higgins",
            "order": 105,
            "profile_path": "/sF69BpL26VOAnYeLbr9RDvpAnW8.jpg"
        },
        {
            "cast_id": 132,
            "character": "Jabba's Henchman (uncredited)",
            "credit_id": "5ae0f0ea925141730a006cca",
            "gender": 2,
            "id": 1490615,
            "name": "Ron Tarr",
            "order": 106,
            "profile_path": "/iMSzEgb5VCREfDQjtxo9cI0wYt5.jpg"
        }
    ],
    "crew": [
        {
            "credit_id": "52fe420dc3a36847f8000437",
            "department": "Directing",
            "gender": 2,
            "id": 1,
            "job": "Director",
            "name": "George Lucas",
            "profile_path": "/7Q5FVw6RhI1gsr1QHmJZuwxshRF.jpg"
        },
        {
            "credit_id": "52fe420dc3a36847f800045b",
            "department": "Production",
            "gender": 2,
            "id": 1,
            "job": "Executive Producer",
            "name": "George Lucas",
            "profile_path": "/7Q5FVw6RhI1gsr1QHmJZuwxshRF.jpg"
        },
        {
            "credit_id": "52fe420dc3a36847f8000461",
            "department": "Production",
            "gender": 2,
            "id": 12401,
            "job": "Producer",
            "name": "Gary Kurtz",
            "profile_path": "/q6tgPiNqzEOIYmHxMrpWoUirmmu.jpg"
        },
        {
            "credit_id": "52fe420dc3a36847f8000467",
            "department": "Production",
            "gender": 0,
            "id": 19801,
            "job": "Producer",
            "name": "Rick McCallum",
            "profile_path": "/qfKgg9sNcIOp2hELpEksqI4DDoo.jpg"
        },
        {
            "credit_id": "52fe420dc3a36847f800046d",
            "department": "Sound",
            "gender": 2,
            "id": 491,
            "job": "Original Music Composer",
            "name": "John Williams",
            "profile_path": "/2Ats98PB1SH2yfEPikiLdhRuXZm.jpg"
        },
        {
            "credit_id": "52fe420dc3a36847f8000473",
            "department": "Camera",
            "gender": 2,
            "id": 7753,
            "job": "Director of Photography",
            "name": "Gilbert Taylor",
            "profile_path": "/in9qOKV8kLCTAO1Zlm0S9dzGEke.jpg"
        },
        {
            "credit_id": "562e75309251414006009955",
            "department": "Writing",
            "gender": 2,
            "id": 1,
            "job": "Writer",
            "name": "George Lucas",
            "profile_path": "/7Q5FVw6RhI1gsr1QHmJZuwxshRF.jpg"
        },
        {
            "credit_id": "56753e6cc3a368167c0026b6",
            "department": "Editing",
            "gender": 2,
            "id": 8425,
            "job": "Editor",
            "name": "Richard Chew",
            "profile_path": null
        },
        {
            "credit_id": "56753e7ec3a368167c0026ba",
            "department": "Editing",
            "gender": 2,
            "id": 10766,
            "job": "Editor",
            "name": "Paul Hirsch",
            "profile_path": null
        },
        {
            "credit_id": "56753e8f92514179db002656",
            "department": "Editing",
            "gender": 1,
            "id": 2551,
            "job": "Editor",
            "name": "Marcia Lucas",
            "profile_path": null
        },
        {
            "credit_id": "56753ea192514179db00265a",
            "department": "Production",
            "gender": 1,
            "id": 1221,
            "job": "Casting",
            "name": "Dianne Crittenden",
            "profile_path": null
        },
        {
            "credit_id": "56753eb192514179db00265c",
            "department": "Production",
            "gender": 1,
            "id": 390,
            "job": "Casting",
            "name": "Irene Lamb",
            "profile_path": null
        },
        {
            "credit_id": "56753ec2c3a368168b0023f2",
            "department": "Production",
            "gender": 2,
            "id": 3176,
            "job": "Casting",
            "name": "Vic Ramos",
            "profile_path": null
        },
        {
            "credit_id": "56753f5292514179d20026b3",
            "department": "Art",
            "gender": 2,
            "id": 568911,
            "job": "Production Design",
            "name": "John Barry",
            "profile_path": null
        },
        {
            "credit_id": "56753f64c3a368167a00263a",
            "department": "Art",
            "gender": 2,
            "id": 5059,
            "job": "Art Direction",
            "name": "Leslie Dilley",
            "profile_path": null
        },
        {
            "credit_id": "56753f7c92514179e3002598",
            "department": "Art",
            "gender": 2,
            "id": 669,
            "job": "Art Direction",
            "name": "Norman Reynolds",
            "profile_path": null
        },
        {
            "credit_id": "56753fbdc3a3681689002463",
            "department": "Art",
            "gender": 2,
            "id": 5058,
            "job": "Set Decoration",
            "name": "Roger Christian",
            "profile_path": null
        },
        {
            "credit_id": "56753fd2c3a368168900246d",
            "department": "Costume & Make-Up",
            "gender": 2,
            "id": 5061,
            "job": "Costume Design",
            "name": "John Mollo",
            "profile_path": null
        },
        {
            "credit_id": "56e9dfbec3a3682d6f0043c7",
            "department": "Visual Effects",
            "gender": 0,
            "id": 1424133,
            "job": "Visual Effects",
            "name": "Paul Huston",
            "profile_path": null
        },
        {
            "credit_id": "588fd62c925141019a00dbfa",
            "department": "Visual Effects",
            "gender": 2,
            "id": 7727,
            "job": "Animation",
            "name": "Phil Tippett",
            "profile_path": "/2uQ0B7fN5cDQk17J1X3pxDSf9y.jpg"
        },
        {
            "credit_id": "5aa9c1a50e0a2664f9003390",
            "department": "Sound",
            "gender": 2,
            "id": 670,
            "job": "Sound Designer",
            "name": "Ben Burtt",
            "profile_path": "/nfkHcZkAIGEyJ0m2jtOKaPyqNxu.jpg"
        },
        {
            "credit_id": "5b04b68bc3a36864b4001b4e",
            "department": "Costume & Make-Up",
            "gender": 2,
            "id": 11837,
            "job": "Makeup Supervisor",
            "name": "Stuart Freeborn",
            "profile_path": null
        },
        {
            "credit_id": "5b04b6b0925141466000272f",
            "department": "Production",
            "gender": 2,
            "id": 66251,
            "job": "Production Manager",
            "name": "Bruce Sharman",
            "profile_path": null
        },
        {
            "credit_id": "5b04b6cdc3a36864900017ce",
            "department": "Production",
            "gender": 2,
            "id": 711,
            "job": "Production Supervisor",
            "name": "Robert Watts",
            "profile_path": "/28BlqdvRGNAdbWuzdXAIYissbgZ.jpg"
        },
        {
            "credit_id": "5b04b75ec3a36864a400219c",
            "department": "Sound",
            "gender": 2,
            "id": 1513636,
            "job": "Production Sound Mixer",
            "name": "Derek Ball",
            "profile_path": null
        },
        {
            "credit_id": "5b04b7720e0a261d8600243b",
            "department": "Sound",
            "gender": 2,
            "id": 1070097,
            "job": "Sound Editor",
            "name": "Gene Corso",
            "profile_path": null
        },
        {
            "credit_id": "5b04b789c3a36864a2002517",
            "department": "Sound",
            "gender": 2,
            "id": 1053,
            "job": "Sound Editor",
            "name": "Gordon Davidson",
            "profile_path": null
        },
        {
            "credit_id": "5b04b7aa9251414652002072",
            "department": "Sound",
            "gender": 0,
            "id": 2048546,
            "job": "Sound Recordist",
            "name": "Michael Galloway",
            "profile_path": null
        },
        {
            "credit_id": "5b04b8dcc3a36864ad00256a",
            "department": "Sound",
            "gender": 2,
            "id": 1399996,
            "job": "Sound Re-Recording Mixer",
            "name": "Robert J. Litt",
            "profile_path": null
        },
        {
            "credit_id": "5b04b8fec3a368649e002031",
            "department": "Sound",
            "gender": 0,
            "id": 1399140,
            "job": "Sound Re-Recording Mixer",
            "name": "Les Fresholtz",
            "profile_path": null
        },
        {
            "credit_id": "5b04b91d0e0a261d9100270d",
            "department": "Sound",
            "gender": 0,
            "id": 74778,
            "job": "Sound Re-Recording Mixer",
            "name": "Don MacDougall",
            "profile_path": null
        },
        {
            "credit_id": "5b04b935c3a3686496002f14",
            "department": "Sound",
            "gender": 0,
            "id": 1446531,
            "job": "Sound Re-Recording Mixer",
            "name": "Bob Minkler",
            "profile_path": null
        },
        {
            "credit_id": "5b04b9440e0a261da5002824",
            "department": "Sound",
            "gender": 2,
            "id": 1399141,
            "job": "Sound Re-Recording Mixer",
            "name": "Michael Minkler",
            "profile_path": null
        },
        {
            "credit_id": "5b04b95b92514146600029ff",
            "department": "Sound",
            "gender": 2,
            "id": 3105,
            "job": "Sound Re-Recording Mixer",
            "name": "Richard Portman",
            "profile_path": null
        },
        {
            "credit_id": "5b04b9bd0e0a261d8f002d04",
            "department": "Sound",
            "gender": 2,
            "id": 1544697,
            "job": "Sound Editor",
            "name": "Robert R. Rutledge",
            "profile_path": null
        },
        {
            "credit_id": "5b04b9f3c3a36864b70033ad",
            "department": "Sound",
            "gender": 2,
            "id": 2036880,
            "job": "Supervising Sound Editor",
            "name": "Sam F. Shaw",
            "profile_path": null
        },
        {
            "credit_id": "5b04ba29c3a36864a2002796",
            "department": "Sound",
            "gender": 2,
            "id": 1446533,
            "job": "Sound Re-Recording Mixer",
            "name": "Ray West",
            "profile_path": null
        },
        {
            "credit_id": "5b04ba51c3a36864b7003430",
            "department": "Costume & Make-Up",
            "gender": 2,
            "id": 2021308,
            "job": "Wardrobe Supervisor",
            "name": "Ron Beck",
            "profile_path": null
        },
        {
            "credit_id": "5ba184cfc3a36871930010e4",
            "department": "Sound",
            "gender": 2,
            "id": 138618,
            "job": "Sound Mix Technician",
            "name": "Gary Rizzo",
            "profile_path": "/skI8AcW5WLBkTjZP4d7g3MMF3LS.jpg"
        }
    ]
}

https://api.themoviedb.org/3/movie/11/keywords?api_key=e4f5b3b79e2ab374455c1c1e1d0ca014

In [ ]:
{
    "id": 11,
    "keywords": [
        {
            "id": 803,
            "name": "android"
        },
        {
            "id": 4270,
            "name": "galaxy"
        },
        {
            "id": 4271,
            "name": "hermit"
        },
        {
            "id": 10013,
            "name": "death star"
        },
        {
            "id": 10016,
            "name": "lightsaber"
        },
        {
            "id": 10527,
            "name": "jedi"
        },
        {
            "id": 11107,
            "name": "rescue mission"
        },
        {
            "id": 11195,
            "name": "empire"
        },
        {
            "id": 11196,
            "name": "rebellion"
        },
        {
            "id": 13194,
            "name": "planet"
        },
        {
            "id": 155336,
            "name": "smuggler"
        },
        {
            "id": 160134,
            "name": "the force"
        },
        {
            "id": 161176,
            "name": "space opera"
        },
        {
            "id": 163295,
            "name": "galactic war"
        },
        {
            "id": 178710,
            "name": "stormtrooper"
        },
        {
            "id": 178712,
            "name": "totalitarianism"
        },
        {
            "id": 242443,
            "name": "star wars"
        }
    ]
}